# FM

In [1]:
# 导入
import sys

sys.path.append("E:/dataFiles/github/MFlow")

In [2]:
# 数据生成
import numpy as np
from sklearn.datasets import make_circles

# 加载同心圆数据
xs, ys = make_circles(600, noise=0.1, factor=0.2)
ys = 2 * ys - 1  # 0/1转换为-1/1

# 特征维数
FEAT_DIM = 20
# 构造噪声特征
xs = np.concatenate([xs, np.random.normal(0.0, 0.01, (600, FEAT_DIM - 2))], axis=1)
# 隐藏向量维度
HIDDEN_DIM = 2

print(xs.shape, ys.shape)
print(xs[0], ys[0])

(600, 20) (600,)
[ 2.59504557e-02 -1.00953177e-01  1.57808557e-02 -4.40830782e-03
  9.16414764e-04 -9.85662887e-03  9.94442073e-03  2.06290276e-02
 -1.22026847e-02 -8.62425970e-03 -9.96615840e-03 -1.96601127e-05
  1.39471189e-03  6.73275504e-03  1.00175070e-03  6.50252432e-03
  5.95354590e-03  5.87400539e-03  5.25179067e-03  2.37681886e-04] 1


In [3]:
# 训练
from mflow import core, ops, opts

# 超参数
lr = 0.001
epoch = 50
batch_size = 16

with core.NameScope("FM"):
    # 初始化变量
    x = core.Variable(size=(FEAT_DIM, 1), trainable=False)
    y = core.Variable(size=(1, 1), trainable=False)
    w = core.Variable(size=(1, FEAT_DIM), trainable=True)
    h = core.Variable(size=(HIDDEN_DIM, FEAT_DIM), trainable=True)
    b = core.Variable(size=(1, 1), trainable=True)
    # 模型定义
    pred = ops.Add(
        ops.MatMal(
            ops.Reshape(x, shape=(1, FEAT_DIM)), 
            ops.MatMal(ops.MatMal(ops.Reshape(h, shape=(FEAT_DIM, HIDDEN_DIM)), h), x)
        ),  # 二次项
        ops.MatMal(w, x),  # 一次项
        b  # 偏置项
    )
    predicter = ops.Logistic(pred)
    loss = ops.loss.LogLoss(ops.Multiply(y, pred))
    adam = opts.Adam(core.DefaultGraph, loss, lr)
    # 开始训练
    for ep in range(1, epoch + 1):
        bs_idx = 0  # 批次计数
        # 这是一个epoch的过程
        for i, (feat, lab) in enumerate(zip(xs, ys)):
            x.setValue(np.mat(feat).T)
            y.setValue(np.mat(lab))
            adam.step()
            bs_idx += 1
            if bs_idx == batch_size:
                adam.update()
                bs_idx = 0
        # 一个epoch完成后进行评估
        preds = []
        for feat in xs:
            x.setValue(np.mat(feat).T)
            predicter.forward()
            preds.append(predicter.value[0, 0])  # 结果
        preds = (np.array(preds) > 0.5).astype("int") * 2 - 1
        acc = (ys == preds).astype("uint8").sum() / len(xs)
        print("Epoch: {:d}, acc: {:.3f}.".format(ep, acc))

Epoch: 1, acc: 0.563.
Epoch: 2, acc: 0.560.
Epoch: 3, acc: 0.552.
Epoch: 4, acc: 0.550.
Epoch: 5, acc: 0.545.
Epoch: 6, acc: 0.545.
Epoch: 7, acc: 0.548.
Epoch: 8, acc: 0.547.
Epoch: 9, acc: 0.547.
Epoch: 10, acc: 0.543.
Epoch: 11, acc: 0.547.
Epoch: 12, acc: 0.545.
Epoch: 13, acc: 0.547.
Epoch: 14, acc: 0.547.
Epoch: 15, acc: 0.547.
Epoch: 16, acc: 0.545.
Epoch: 17, acc: 0.542.
Epoch: 18, acc: 0.542.
Epoch: 19, acc: 0.547.
Epoch: 20, acc: 0.548.
Epoch: 21, acc: 0.548.
Epoch: 22, acc: 0.548.
Epoch: 23, acc: 0.548.
Epoch: 24, acc: 0.547.
Epoch: 25, acc: 0.547.
Epoch: 26, acc: 0.548.
Epoch: 27, acc: 0.547.
Epoch: 28, acc: 0.548.
Epoch: 29, acc: 0.550.
Epoch: 30, acc: 0.552.
Epoch: 31, acc: 0.552.
Epoch: 32, acc: 0.553.
Epoch: 33, acc: 0.550.
Epoch: 34, acc: 0.552.
Epoch: 35, acc: 0.552.
Epoch: 36, acc: 0.552.
Epoch: 37, acc: 0.552.
Epoch: 38, acc: 0.552.
Epoch: 39, acc: 0.552.
Epoch: 40, acc: 0.553.
Epoch: 41, acc: 0.552.
Epoch: 42, acc: 0.550.
Epoch: 43, acc: 0.548.
Epoch: 44, acc: 0.54